<a href="https://colab.research.google.com/github/papagorgio23/Python101/blob/master/Survivor_Pool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![A.I. Sports](https://raw.githubusercontent.com/papagorgio23/Python101/master/newlogo.png)

www.AISportsFirm.com

# NFL Survivor Pool Optimizer

![Draft Kings](https://raw.githubusercontent.com/papagorgio23/Python101/master/DraftKings.png)

We are going to tap into the Draft Kings API to access player prices and average player scores. 

We are then going to use this information to create optimal lineups. 

*   Optimial lineup = Combination of players that will score the most points


## Linear Programming (The Math)

The basic framework of linear/integer programming is as follows:


*   There are **Decision Variables**
    *  These will be the variables that we can select (i.e. which team to pick to win each week)
*   There is an **Objective Function** to either Maximize or Minimize
    * Our goal will be to Maximize the win probability
*   There are **Constraints** we must abide by
    * Only select a team once, non-negativity




## Definitions:

<br>

**Index Sets:**

Let ***I*** be the set of Weeks in the NFL season, indexed by *i*

Let ***J*** be the set of NFL teams, indexed by *j*


<br>

**Variables:**

Let ***x*** be the team

Let ***P*** be the win probability for each matchup

Let ***W*** be the week number in the season


<br>

**Other Variables:**

Let ***n*** be the total number of teams

Let ***m*** be the total number of weeks

<br>

***

<br>

$x_{ij} = Team\ Selected$

<br>

$P_{ij} = Probability\ to\ win$

<br>


***

<br>


#### implicit Form:


$$maximize\ \ \ \ \ \ z = \sum_{i=1}^n\sum_{j=1}^m P_{ij} * x_{ij} $$

<br>

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ subject\ to\ \ \ \ \ \sum_{i=1}^n\sum_{j=1}^m C_{ij} * x_{ij} \leq Max\ Salary$$


$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = QB$$


$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij}  =2, \ \ for\ \ i = RB$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =3, \ \ for\ \ i = WR$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = TE$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = FLEX$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = DST$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{i=1}^n\sum_{j=1}^m x_{ij} = 9$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ x_{ij} \geq 0,\ \forall \ i,\ j$$

<br>

***

## The Dream Survivor Pool Rules

<br>

<br>

Basic Game-play :It's simple... make 1 pick each week, that's it!
Once you pick a team, you can't pick them again, regardless if they win or lose. 

After you get two losses you are out unless you decide to buy-back in ($5) or join the consolation pool (If you lose twice prior to week 8 picks). Each loss after buying back in requires a buy-back to continue.

<br>

**Advanced Game-play**:  There are (3) exciting twists to keep in mind:

<br>

1.   Loser's Week - Week 7

     *   In Week 7, you must choose a team to lose.
     *   **BUT**, it has to be one of the (6) teams you selected in Weeks 1 - 6.
     *   They do not need to have actually won previously to be eligble.

<br>

2.   Two-pick Week - Week 14

     *   In Week 14, you must select (2) teams to win.
     *   Only getting both right will get you a win in the standings. 
     *   Any other result (outside of two ties) will get you a loss.

<br>

3.   Pre-selected double-dip

     *   Nominate one NON playoff team from 2018 to be your Double-Dip for 2019
     *   This team may be selected twice this year with no restrictions
     *   If you also pick this team in Weeks 1-6, you can pick them as your loser and use them 3 times
     *   Nominations will be completed in the first pick survey.




[Official Rules](https://docs.google.com/spreadsheets/d/1ePUp4tcafBpYBrkmc7BRlzZtlKx3J06dlENEzThovdM/edit#gid=1377458333)

# Python Code

### Load Libraries

In [ ]:
!pip install pulp

import urllib, json
import pandas as pd
import numpy as np
import re

from pulp import *

     |████████████████████████████████| 13.6MB 4.8MB/s 
  Created wheel for pulp: filename=PuLP-1.6.10-cp36-none-any.whl size=12269903 sha256=aae9fe861bb643c371031130798c4f26b3acf6b7bb916bd86103ac57b37b5f65
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp


## Get Data

In [ ]:
survivor = pd.read_csv("https://raw.githubusercontent.com/papagorgio23/Python101/master/survivor.csv")
survivor.head()

,team,week,Prob
0,ARZ,1,0.468766
1,ATL,1,0.394124
2,BAL,1,0.731833
3,BUF,1,0.431858
4,CAR,1,0.238872


## Change Week 7 Probabilities to get Loss Probability

In [ ]:
survivor['Prob'] = np.where(survivor['week'] == 7, 1 - survivor['Prob'], survivor['Prob'])

In [ ]:
survivor['week'] = survivor.apply(lambda x: "Week %s" % x['week'], axis = 1)

# Define The Problem

## Prepare for LP formula


The data needs to be transformed slightly differently for PuLP to work with it.


<br>

$P_{ij} = Probability\ of\ Win$

<br>

$x_{ij} = Team\ Selected$

<br>

In [ ]:
# create 2 dictionaries: one containing the salaries of each player, the other containing the points of each player

# Win Probabilities for each matchup (P_ij)
winProbs = {} 

# loop through each position and appened player name, salary, points to correct dictionary
for week in survivor["week"].unique():
    survivor_week = survivor[survivor["week"] == week]
    winProb = list(survivor_week[["team","Prob"]].set_index("team").to_dict().values())[0]
    winProbs[week] = winProb

In [ ]:
# Points of Player (X_ij)
teams = {} 

# loop through each position and appened player name, salary, points to correct dictionary
for team in survivor["team"].unique():
    survivor_team = survivor[survivor["team"] == team]
    team_list = list(survivor_team[["team","week"]].set_index("week").to_dict().values())[0]
    teams[team] = team_list

## Position Constraints

<br>

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = QB$$


$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij}  =2, \ \ for\ \ i = RB$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =3, \ \ for\ \ i = WR$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = TE$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = FLEX$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = DST$$

<br>

Flex position can be either a RB, WR, or TE. 

We will run 3 iterations with the flex being each on of these positions.

In [ ]:
# Team Constraints
team_num_available = {
    "ARZ": 1,
    "ATL": 2,
    "BAL": 1,
    "BUF": 1,
    "CAR": 1,
    "CHI": 1,
    "CIN": 1,
    "CLE": 1,
    "DAL": 1,
    "DEN": 1,
    "DET": 1,
    "GB": 1,
    "HOU": 1,
    "IND": 1,
    "JAX": 1,
    "KC": 1,
    "LAC": 1,
    "LAR": 1,
    "MIA": 1,
    "MIN": 1,
    "NE": 1,
    "NO": 1,
    "NYG": 1,
    "NYJ": 1,
    "OAK": 1,
    "PHI": 1,
    "PIT": 1,
    "SEA": 1,
    "SF": 1,
    "TB": 1,
    "TEN": 1,
    "WAS": 1
}

In [ ]:
# Team Constraints
week_num_available = {
    "Week 1": 1,
    "Week 2": 1,
    "Week 3": 1,
    "Week 4": 1,
    "Week 5": 1,
    "Week 6": 1,
    "Week 7": 1,
    "Week 8": 1,
    "Week 9": 1,
    "Week 10": 1,
    "Week 11": 1,
    "Week 12": 1,
    "Week 13": 1,
    "Week 14": 2,
    "Week 15": 1,
    "Week 16": 1,
    "Week 17": 1
}

## Create Player Variables

<br>

$x_{ij} = Player$


<br>

In [ ]:
# Create the variables
_vars = {i: LpVariable.dict(i, j, cat="Binary") for i, j in winProbs.items()}

In [ ]:
# Create the variables
vars = {i: LpVariable.dict(i, j, cat="Binary") for i, j in teams.items()}

In [ ]:
_vars.values()

dict_values([{'ARZ': Week_1_ARZ, 'ATL': Week_1_ATL, 'BAL': Week_1_BAL, 'BUF': Week_1_BUF, 'CAR': Week_1_CAR, 'CHI': Week_1_CHI, 'CIN': Week_1_CIN, 'CLE': Week_1_CLE, 'DAL': Week_1_DAL, 'DEN': Week_1_DEN, 'DET': Week_1_DET, 'GB': Week_1_GB, 'HOU': Week_1_HOU, 'IND': Week_1_IND, 'JAX': Week_1_JAX, 'KC': Week_1_KC, 'LAC': Week_1_LAC, 'LAR': Week_1_LAR, 'MIA': Week_1_MIA, 'MIN': Week_1_MIN, 'NE': Week_1_NE, 'NO': Week_1_NO, 'NYG': Week_1_NYG, 'NYJ': Week_1_NYJ, 'OAK': Week_1_OAK, 'PHI': Week_1_PHI, 'PIT': Week_1_PIT, 'SEA': Week_1_SEA, 'SF': Week_1_SF, 'TB': Week_1_TB, 'TEN': Week_1_TEN, 'WAS': Week_1_WAS}, {'ARZ': Week_2_ARZ, 'ATL': Week_2_ATL, 'BAL': Week_2_BAL, 'BUF': Week_2_BUF, 'CAR': Week_2_CAR, 'CHI': Week_2_CHI, 'CIN': Week_2_CIN, 'CLE': Week_2_CLE, 'DAL': Week_2_DAL, 'DEN': Week_2_DEN, 'DET': Week_2_DET, 'GB': Week_2_GB, 'HOU': Week_2_HOU, 'IND': Week_2_IND, 'JAX': Week_2_JAX, 'KC': Week_2_KC, 'LAC': Week_2_LAC, 'LAR': Week_2_LAR, 'MIA': Week_2_MIA, 'MIN': Week_2_MIN, 'NE': Week_2

In [ ]:
_vars.items()

dict_items([('Week 1', {'ARZ': Week_1_ARZ, 'ATL': Week_1_ATL, 'BAL': Week_1_BAL, 'BUF': Week_1_BUF, 'CAR': Week_1_CAR, 'CHI': Week_1_CHI, 'CIN': Week_1_CIN, 'CLE': Week_1_CLE, 'DAL': Week_1_DAL, 'DEN': Week_1_DEN, 'DET': Week_1_DET, 'GB': Week_1_GB, 'HOU': Week_1_HOU, 'IND': Week_1_IND, 'JAX': Week_1_JAX, 'KC': Week_1_KC, 'LAC': Week_1_LAC, 'LAR': Week_1_LAR, 'MIA': Week_1_MIA, 'MIN': Week_1_MIN, 'NE': Week_1_NE, 'NO': Week_1_NO, 'NYG': Week_1_NYG, 'NYJ': Week_1_NYJ, 'OAK': Week_1_OAK, 'PHI': Week_1_PHI, 'PIT': Week_1_PIT, 'SEA': Week_1_SEA, 'SF': Week_1_SF, 'TB': Week_1_TB, 'TEN': Week_1_TEN, 'WAS': Week_1_WAS}), ('Week 2', {'ARZ': Week_2_ARZ, 'ATL': Week_2_ATL, 'BAL': Week_2_BAL, 'BUF': Week_2_BUF, 'CAR': Week_2_CAR, 'CHI': Week_2_CHI, 'CIN': Week_2_CIN, 'CLE': Week_2_CLE, 'DAL': Week_2_DAL, 'DEN': Week_2_DEN, 'DET': Week_2_DET, 'GB': Week_2_GB, 'HOU': Week_2_HOU, 'IND': Week_2_IND, 'JAX': Week_2_JAX, 'KC': Week_2_KC, 'LAC': Week_2_LAC, 'LAR': Week_2_LAR, 'MIA': Week_2_MIA, 'MIN': We

## Solve Problem

In [ ]:
team_num_available

{'ARZ': 1,
 'ATL': 2,
 'BAL': 1,
 'BUF': 1,
 'CAR': 1,
 'CHI': 1,
 'CIN': 1,
 'CLE': 1,
 'DAL': 1,
 'DEN': 1,
 'DET': 1,
 'GB': 1,
 'HOU': 1,
 'IND': 1,
 'JAX': 1,
 'KC': 1,
 'LAC': 1,
 'LAR': 1,
 'MIA': 1,
 'MIN': 1,
 'NE': 1,
 'NO': 1,
 'NYG': 1,
 'NYJ': 1,
 'OAK': 1,
 'PHI': 1,
 'PIT': 1,
 'SEA': 1,
 'SF': 1,
 'TB': 1,
 'TEN': 1,
 'WAS': 1}

In [ ]:
for i, x in _vars.items():
  print([winProbs[i][j] * _vars[i][j] for j in x])

[0.46876566106252904*Week_1_ARZ + 0.0, 0.394123615126611*Week_1_ATL + 0.0, 0.731833130614707*Week_1_BAL + 0.0, 0.431858277694575*Week_1_BUF + 0.0, 0.23887227742780104*Week_1_CAR + 0.0, 0.7255862135147719*Week_1_CHI + 0.0, 0.310417185301878*Week_1_CIN + 0.0, 0.606654250299282*Week_1_CLE + 0.0, 0.5445757600110379*Week_1_DAL + 0.0, 0.734839741802024*Week_1_DEN + 0.0, 0.531234338937471*Week_1_DET + 0.0, 0.27441378648522796*Week_1_GB + 0.0, 0.436151304830065*Week_1_HOU + 0.0, 0.457830217619199*Week_1_IND + 0.0, 0.159505799010328*Week_1_JAX + 0.0, 0.840494200989672*Week_1_KC + 0.0, 0.542169782380801*Week_1_LAC + 0.0, 0.7611277225721991*Week_1_LAR + 0.0, 0.268166869385293*Week_1_MIA + 0.0, 0.605876384873389*Week_1_MIN + 0.0, 0.647831496128254*Week_1_NE + 0.0, 0.5638486951699351*Week_1_NO + 0.0, 0.45542423998896203*Week_1_NYG + 0.0, 0.568141722305425*Week_1_NYJ + 0.0, 0.265160258197976*Week_1_OAK + 0.0, 0.711332531858839*Week_1_PHI + 0.0, 0.3521685038717461*Week_1_PIT + 0.0, 0.6895828146981221

In [ ]:
max(winProbs, key=lambda k: winProbs[k])
for i, x in _vars.items():
    print()

TypeError: ignored

In [ ]:
_vars.get("Week 2")["ARZ"]

Week_2_ARZ

In [ ]:
for i in range(1,18):
  week_num = f"Week {i}"
  try:
      print(lpSum(_vars.get(week_num)["ARZ"]))
  except KeyError:
      continue
  #print(_vars.get(week_num)["ARZ"])
  #print(week_num)

Week_1_ARZ
Week_2_ARZ
Week_3_ARZ
Week_4_ARZ
Week_5_ARZ
Week_6_ARZ
Week_7_ARZ
Week_8_ARZ
Week_9_ARZ
Week_10_ARZ
Week_11_ARZ
Week_13_ARZ
Week_14_ARZ
Week_15_ARZ
Week_16_ARZ
Week_17_ARZ


In [ ]:
for i, x in _vars.items():
  #print([1 * _vars[i][j] for j in x])
  print(_vars.get("Week 17"))

{'ARZ': Week_17_ARZ, 'ATL': Week_17_ATL, 'BAL': Week_17_BAL, 'BUF': Week_17_BUF, 'CAR': Week_17_CAR, 'CHI': Week_17_CHI, 'CIN': Week_17_CIN, 'CLE': Week_17_CLE, 'DAL': Week_17_DAL, 'DEN': Week_17_DEN, 'DET': Week_17_DET, 'GB': Week_17_GB, 'HOU': Week_17_HOU, 'IND': Week_17_IND, 'JAX': Week_17_JAX, 'KC': Week_17_KC, 'LAC': Week_17_LAC, 'LAR': Week_17_LAR, 'MIA': Week_17_MIA, 'MIN': Week_17_MIN, 'NE': Week_17_NE, 'NO': Week_17_NO, 'NYG': Week_17_NYG, 'NYJ': Week_17_NYJ, 'OAK': Week_17_OAK, 'PHI': Week_17_PHI, 'PIT': Week_17_PIT, 'SEA': Week_17_SEA, 'SF': Week_17_SF, 'TB': Week_17_TB, 'TEN': Week_17_TEN, 'WAS': Week_17_WAS}
{'ARZ': Week_17_ARZ, 'ATL': Week_17_ATL, 'BAL': Week_17_BAL, 'BUF': Week_17_BUF, 'CAR': Week_17_CAR, 'CHI': Week_17_CHI, 'CIN': Week_17_CIN, 'CLE': Week_17_CLE, 'DAL': Week_17_DAL, 'DEN': Week_17_DEN, 'DET': Week_17_DET, 'GB': Week_17_GB, 'HOU': Week_17_HOU, 'IND': Week_17_IND, 'JAX': Week_17_JAX, 'KC': Week_17_KC, 'LAC': Week_17_LAC, 'LAR': Week_17_LAR, 'MIA': Week_17

In [ ]:
# initialize the problem statement (Maximize)
prob = LpProblem("Survivor Pool Optimizer", LpMaximize)

# initialize lists to hold constraints
winProb = []
week = []

# Setting up the reward
for i, x in _vars.items():
    #week += lpSum([teams[i][j] * _vars[i][j] for j in x])
    winProb += lpSum([winProbs[i][j] * _vars[i][j] for j in x])
    #prob += lpSum([teams[i][j] * _vars[i][j] for j in x]) <= team_num_available[i]
    prob += lpSum([_vars[i][j] for j in x]) <= week_num_available[i]
    #prob += lpSum([_vars[i][k] for k in i) <= team_num_available[i]


# Setting up the reward
for i, x in vars.items():
    prob += lpSum([vars[i][k] for k in x]) <= team_num_available[i]

#prob += lpSum(["Week_1_KC" + 'Week_2_KC'] <= 1)
# Salary Constraint
#prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(winProb)

In [ ]:
for i, x in _vars.items():
    print(_vars[3])
    #print(lpSum([_vars[i][j] for j in x]) <= week_num_available[i])

KeyError: ignored

In [ ]:
# Setting up the reward
for i, x in vars.items():
    print(lpSum([vars[i][k] for k in x]) <= team_num_available[i])

ARZ_Week_1 + ARZ_Week_10 + ARZ_Week_11 + ARZ_Week_13 + ARZ_Week_14 + ARZ_Week_15 + ARZ_Week_16 + ARZ_Week_17 + ARZ_Week_2 + ARZ_Week_3 + ARZ_Week_4 + ARZ_Week_5 + ARZ_Week_6 + ARZ_Week_7 + ARZ_Week_8 + ARZ_Week_9 <= 1
ATL_Week_1 + ATL_Week_10 + ATL_Week_11 + ATL_Week_12 + ATL_Week_13 + ATL_Week_14 + ATL_Week_15 + ATL_Week_16 + ATL_Week_17 + ATL_Week_2 + ATL_Week_3 + ATL_Week_4 + ATL_Week_5 + ATL_Week_6 + ATL_Week_7 + ATL_Week_8 <= 2
BAL_Week_1 + BAL_Week_10 + BAL_Week_11 + BAL_Week_12 + BAL_Week_13 + BAL_Week_14 + BAL_Week_15 + BAL_Week_16 + BAL_Week_17 + BAL_Week_2 + BAL_Week_3 + BAL_Week_4 + BAL_Week_5 + BAL_Week_6 + BAL_Week_7 + BAL_Week_9 <= 1
BUF_Week_1 + BUF_Week_10 + BUF_Week_11 + BUF_Week_12 + BUF_Week_13 + BUF_Week_14 + BUF_Week_15 + BUF_Week_16 + BUF_Week_17 + BUF_Week_2 + BUF_Week_3 + BUF_Week_4 + BUF_Week_5 + BUF_Week_7 + BUF_Week_8 + BUF_Week_9 <= 1
CAR_Week_1 + CAR_Week_10 + CAR_Week_11 + CAR_Week_12 + CAR_Week_13 + CAR_Week_14 + CAR_Week_15 + CAR_Week_16 + CAR_Week_17 + 

In [ ]:
# Setting up the reward
for i, x in _vars.items():
    print(lpSum([_vars[i][k] for k in x])) #<= team_num_available[i])

KeyError: ignored

In [ ]:
min(prices, key=lambda k: prices[k])

In [ ]:
prob.solve()

1

In [ ]:
# my beautiful function to display the results of our LP
def summary(prob):
    div = '\n-----------------------------------------\n'
    print("\n-----------------------------------------")
    print("Linear Programming Optimization Solution:")
    print("-----------------------------------------\n")
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Salary:\n")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:\n")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))
    print(div)
    #print("Total Cost:  ${}".format(round(eval(constraint_pretty))))
    print("Expected Win Total: {}\n".format(round(eval(score),2)))

In [ ]:
summary(prob)


-----------------------------------------
Linear Programming Optimization Solution:
-----------------------------------------

Variables:

Week_10_LAC = 1.0
Week_11_NO = 1.0
Week_12_NE = 1.0
Week_13_KC = 1.0
Week_14_LAC = 1.0
Week_14_NO = 1.0
Week_15_BAL = 1.0
Week_16_LAR = 1.0
Week_17_NE = 1.0
Week_1_KC = 1.0
Week_2_KC = 1.0
Week_3_CHI = 1.0
Week_4_KC = 1.0
Week_5_CHI = 1.0
Week_6_LAR = 1.0
Week_7_NYJ = 1.0
Week_8_KC = 1.0
Week_9_KC = 1.0

-----------------------------------------

Salary:


-----------------------------------------

Score:

0.8424346043273551*1.0 + 0.7757088920979391*1.0 + 0.721627070178072*1.0 + 0.89689121539152*1.0 + 0.793726115014573*1.0 + 0.799931307715494*1.0 + 0.739906691787478*1.0 + 0.8140903716766469*1.0 + 0.7767250119768259*1.0 + 0.840494200989672*1.0 + 0.879831088570313*1.0 + 0.77712145748105*1.0 + 0.8599291022146751*1.0 + 0.8018707345836191*1.0 + 0.805739698163545*1.0 + 0.7575318134664171*1.0 + 0.8281947733146191*1.0 + 0.7606250196862809*1.0 = 14.47237916

## Flex Postion = RB

In [ ]:
# salary and points
salaries = {} 
points = {} 
for pos in availables["position"].unique():
    available_pos = availables[availables["position"] == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

# Max Salary limit
SALARY_CAP = 50000


# Position Constraints
pos_num_available = {
    "QB": 1,
    "RB": 3, # FLEX POSITION = RB
    "WR": 3, 
    "TE": 1, 
    "DST": 1
}



# initialize the problem statement (Maximize)
prob = LpProblem("Fantasy Football Lineup", LpMaximize)

# Create the variables
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

# initialize lists to hold constraints
score = []
salary = []

# Setting up the reward
for k, v in _vars.items():
    salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    score += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    

# Salary Constraint
prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(score)

prob.solve()

summary(prob)

## Flex Postion = WR

In [ ]:
# salary and points
salaries = {} 
points = {} 
for pos in availables["position"].unique():
    available_pos = availables[availables["position"] == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

# Max Salary limit
SALARY_CAP = 50000


# Position Constraints
pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 4, # FLEX POSITION = WR
    "TE": 1, 
    "DST": 1
}



# initialize the problem statement (Maximize)
prob = LpProblem("Fantasy Football Lineup", LpMaximize)

# Create the variables
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

# initialize lists to hold constraints
score = []
salary = []

# Setting up the reward
for k, v in _vars.items():
    salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    score += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    

# Salary Constraint
prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(score)

prob.solve()

summary(prob)

## Flex Postion = TE

In [ ]:
# salary and points
salaries = {} 
points = {} 
for pos in availables["position"].unique():
    available_pos = availables[availables["position"] == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

# Max Salary limit
SALARY_CAP = 50000


# Position Constraints
pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 3,
    "TE": 2, # FLEX POSITION = TE
    "DST": 1
}



# initialize the problem statement (Maximize)
prob = LpProblem("Fantasy Football Lineup", LpMaximize)

# Create the variables
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

# initialize lists to hold constraints
score = []
salary = []

# Setting up the reward
for k, v in _vars.items():
    salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    score += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    

# Salary Constraint
prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(score)

prob.solve()

summary(prob)